In [14]:
%matplotlib inline
#standard imports 
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# There are three (obligatory) problems to solve due 6.11

1.k-NN Problem

2.Bayes value Problem

3.Expectated value Problem

# 1. The first problem should be solved according to the guidelines from

https://github.com/wkrzemien/dataScienceAndML2020/blob/master/notebooks/knn/knn_first.ipynb

# 1. KNN algorithm has three steps:

(1) Calculate Euclidean Distance.

(2) Get Nearest Neighbors.

(3)  Make Predictions.


File available here: http://koza.if.uj.edu.pl/~krzemien/machine_learning2021/materials/datasets/iris_data.csv

In [15]:
input_file = 'iris_data.csv'

In [18]:
data = pd.read_csv(input_file)

FileNotFoundError: [Errno 2] No such file or directory: 'iris_data.csv'

In [ ]:
data.head()

In [17]:
data.info()

NameError: name 'data' is not defined

In [ ]:
data.describe()

In [ ]:
data['species'].unique()

In [ ]:
dataSetosa=data[data['species']=='Iris-setosa']
dataVersicolor=data[data['species']=='Iris-versicolor']
dataVirginica=data[data['species']=='Iris-virginica']
datasets =[dataSetosa,dataVersicolor,dataVirginica]

In [ ]:
data.groupby('species').mean()

In [ ]:
data.groupby('species').std()

In [ ]:
def plotDatasets(datasets, dataSetsLabels, columnsToPlot=None, xLabel='x', yLabel='y', ):
  """
    Args:
      columnsToPlot(list): list of numbers corresponding to columns that should be plotted.
                           e.g. if we want to plot 3rd column  vs 4th column columnsToPlot should be
                           a list [2,3]. By default it is 1rst vs 2nd so [0,1]

  """
  colors = ['red','green','blue'] 
  if not columnsToPlot:
    columnsToPlot = [0, 1]
  for d, dlabel, color in zip(datasets, dataSetsLabels, colors):
    xdata = d.iloc[:, columnsToPlot[0]]
    ydata = d.iloc[:, columnsToPlot[1]]
    plt.scatter(xdata, ydata, label=dlabel, color = color)
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
  plt.legend()
  plt.show()
  # plt.savefig('plot.png') #save figure to file

In [ ]:
dLabels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [ ]:
plotDatasets(datasets, dLabels, [0, 1], 'sepal_length', 'sepal_width')

In [ ]:
plotDatasets(datasets, dLabels, [2, 3], 'petal_length', 'petal_width')

In [ ]:
def distance(x,y):
    return -1

In [ ]:
from operator import itemgetter

def getNeighbours(x, Xtrain, Ytrain, metric=distance):
  """
    Function should return neighbours of x, sorted by distance from x. 
    Xtrain and Ytrain form the training set e.g. Xtrain= [[0,1], [10,10]] Y=[0,1]
    can be interpreted in the following way: we have two points in the feature space: [0,1] and [10,10]
    First point belongs to the class 0 and the second to the class 1.

    In order to sort your results by distance one can use sorted construct.
    E.g. if we have a list of elements each of elements being the list with 2 elements, then we can sort the main list using e.g. the second element of every object:
    testList =[[0,1], [3,0], [2,-1]]   
    resultList = sorted(testList, key = itemgetter(1))  # 1 means second element of every 
    object on the list
    We will end up with: [[2,-1], [3,0], [0,1]]

    Args: 
      x(list): list of features(numbers) which represents a point in the feature space with respect to which we calculate the distance.
      Xtrain(list): of feature vectors which form our training sample e.g. [[0,1], [10,10]] -> two points [0,1] and [10,10] with two features each
      Ytrain(list): of class labels which assing given feature vector (point) to given class e.g. [0,1] the first point belongs to class 0 and the second to 1
      metric(function): that calculates distance between two points from the feature space.
  """
  return (-1, [0, 0], 0)

In [ ]:

def getKNNeighbours(neighbours, k):
  """
    Function should return k nearest neighbours assuming that neighbours are already sorted by distance.

    Args:
      neighbours(list): neighbours sorted by distance e.g. [[1, [2,0],1],  [5, [2,0],1]],
                        where [5,[2,0],1] -> distance 5, closest feature point [2,0] which belongs to the class 1
      k(integer): number of neighbours to return
    Returns:
      list of k nearest neighbours assuming that neighbours are already sorted by distance.
  """
  return [-1, [0, 0], 0]

In [ ]:
def majorityVote(neighbours):
  """
    Args:
      neighbours(list): neighbours sorted by distance e.g. [[1, [2,0],1],  [5, [2,0],1]],
                        where [5,[2,0],1] -> distance 5, closest feature point [2,0] which belongs to the class 1
    Returns:
      number: being the average of class labels  
  """
  return -1.

In [ ]:
def predict(x, Xtrain, Ytrain, k):
  """
    Args:
      x(list): feature point that should be classify
      Xtrain(list): of feature vectors which form our training sample e.g. [[0,1], [10,10]] -> two points [0,1] and [10,10] with two features each
      Ytrain(list): of class labels which assing given feature vector (point) to given class e.g. [0,1] the first point belongs to class 0 and the second to 1
    Returns:
     int:  prediction
  """
  return -1


def predictList(xObjects, Xtrain, Ytrain, k):
  """
    Args:
      xObjects(list): list of feature points that should be classify e.g. [[1,0], [2,2]] -> two points with 2 feature each
      Xtrain(list): of feature vectors which form our training sample e.g. [[0,1], [10,10]] -> two points [0,1] and [10,10] with two features each
      Ytrain(list): of class labels which assing given feature vector (point) to given class e.g. [0,1] the first point belongs to class 0 and the second to 1
    Returns:
     list:  list of int corresponding to predictions
  """
  fakePredictions = [0] * len(xObjects)
  return fakePredictions

In [ ]:
def meanSquaredError(v, w):
  """
    Args:
      v(list): list of numbers
      w(list): list of numbers
    Returns:
     float: mean squared error
  """
  return -1.

In [ ]:
def plotErrors(x, train,valid):
  plt.plot(x, train, label='training error')
  plt.plot(x, valid, label='test error')
  plt.xlabel('k')
  plt.legend()
  plt.show()
  #plt.savefig('errors.png') #save figure to file

In [ ]:
def divideData(dataSet, fraction):
  nbTotal = len(ddataSet.index)
  nbTrain = int(nbTotal*fraction)
  return (dataSet.iloc[:nbTrain,:], dataSet.iloc[nbTrain:,:])

In [ ]:
def runTests():
  # some tests of distance function
  np.testing.assert_almost_equal(distance([1, 1], [1, 1]), 0)
  np.testing.assert_almost_equal(distance([2, 0, 1], [5, 0, 1]), 3)
  np.testing.assert_almost_equal(distance([0, 0], [2, 2]), np.sqrt(8))

  # some tests of getNeighbours function
  Xtrain = [[2, 0], [0, 0],  [1, 0]]
  Ytrain = [1, 0, 1]
  x = [-1, 0]  # with respect to x we calculate the distance
  result = getNeighbours(x, Xtrain, Ytrain)
  expected = ((1, [0, 0], 0), (2, [1, 0], 1), (3, [2, 0], 1))
  np.testing.assert_equal(result, expected)

  # some tests of getKNNeighbours function
  data = [[1, [0, 0], 0], [2, [1, 0], 1], [3, [2, 0], 1]]
  np.testing.assert_equal(getKNNeighbours(data,  k=1), [[1, [0, 0], 0]])
  np.testing.assert_equal(getKNNeighbours(data,  k=2), [
                          [1, [0, 0], 0], [2, [1, 0], 1]])

  # some tests of majorityVote function
  data = [[1, [0, 0], 0], [2, [1, 0], 1], [3, [2, 0], 1]]
  np.testing.assert_equal(majorityVote(data), 2./3.)

  # some tests of predict function
  xTrain = [[0, 0], [1, 0], [2, 0]]
  yTrain = [0, 1, 1]
  x = [-1, 0]
  k = 1
  np.testing.assert_almost_equal(predict(x, xTrain, yTrain, k), 0)
  k = 2
  np.testing.assert_almost_equal(predict(x, xTrain, yTrain, k), 1)
  k = 3
  np.testing.assert_almost_equal(predict(x, xTrain, yTrain, k), 1)

  # some tests of predictList function
  xTrain = [[0, 0], [1, 0], [2, 0]]
  yTrain = [0, 1, 1]
  xToClassify = [[-1, 0], [3, 0]]
  k = 1
  np.testing.assert_almost_equal(predictList(xToClassify, xTrain, yTrain, k), [0., 1.])

In [ ]:
runTests()

In [ ]:
class kNN(object):
    def __init__(self, n_neighbours):
        self.n_neighbours = n_neighbours
        self.Xtrain = []
        self.Ytrain = []
    def fit(self, x,y):
        self.Xtrain = x
        self.Ytrain = y
    def predict(self, Xobjects):
        return predictList(Xobjects,self.Xtrain, self.Ytrain, self.n_neighbours)

In [ ]:
xTrain = [[0, 0], [1, 0], [2, 0]]
yTrain = [0, 1, 1]
xToClassify = [[-1, 0], [3, 0]]
k = 1
model = kNN(k)
model.fit(xTrain,yTrain)
np.testing.assert_almost_equal(model.predict(xToClassify), [0., 1.])

In [ ]:
classLabels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
columnLabels = ['sepal_length', 'sepal_width',
                  'petal_length', 'petal_width', 'species']
# First let's try  Iris-setosa vs Iris-versicolor, two features sepal-length vs sepal-width
# we get rid of the third class Iris-viriginica
dataWithoutVirignica = data[data['species'] != 'Iris-virginica']

def toNumeric(row):
    if row.species == 'Iris-versicolor':
        return 0
    else:
        return 1

dataWithoutVirignica['class']=dataWithoutVirignica.apply(lambda row:toNumeric(row),axis=1)
  
# we leave only sepal_length, sepal_width and class label columns
columnsToLeave=['sepal_length', 'sepal_width', 'class']  
dataSepal = dataWithoutVirignica[columnsToLeave]

In [ ]:
fraction = 0.8
# we  divide the content into the training and validation set
# we also randomly shuffle the content
trainingSet=dataSepal.sample(frac=fraction)
validationSet=dataSepal.drop(trainingSet.index)
xTrain = trainingSet.iloc[:, 1:].values.tolist()
yTrain = trainingSet.iloc[:, 0].values.tolist()
xValid = validationSet.iloc[:, 1:].values.tolist()
yValid = validationSet.iloc[:, 0].values.tolist()

In [ ]:
if fraction < 1:
    #we calculate training error and validation error
    trainError=[]
    validError=[]
    kRange =range(1,80)
    trainPredictions = predictList(xTrain, xTrain, yTrain, 1)
    for k in kRange:
      trainPredictions = predictList(xTrain, xTrain, yTrain, k)
      validationPredictions = predictList(xValid, xTrain, yTrain, k)
      trainError.append(meanSquaredError(trainPredictions,yTrain))
      validError.append(meanSquaredError(validationPredictions,yValid))

In [ ]:
plotErrors(kRange, trainError,validError)

# 2. Bayes value

A pharmaceutical company developed a test for detecting the rare disease, which is carried by 0.5 % cases of the whole population. Let’s assume that the test gives positive results for 96% of the cases if the patient is ill, but it also gives positive results in 5% of the healthy patient. What is the probability that a patient is ill if his test gave a positive result?

The natural solution to this problem is (almost the same as https://www.youtube.com/watch?v=R13BD8qKeTg):
    
$$P(\text{ill}|P)=\frac{P(P|\text{ill})\cdot P(\text{ill})}{P(ill) \cdot P(P|ill)+P(-ill) \cdot P(P|-ill)}.$$

$$P(\text{ill})=0.005$$

$$P(-\text{ill})=0.995$$

$$P(P|\text{ill})=TP=0.960$$

$$P(P|\text{-ill})=FP=0.050$$

In [10]:
#Calculations
P_ill=0.005
P_not_ill=0.995
P_P_ill=0.960
P_P_not_ill=0.050

P_ill_P=((P_P_ill*P_ill)/(P_ill*P_P_ill+P_not_ill*P_P_not_ill))

print("The probability that the patient is ill on condition that they were positively diagnosed is: {:.4g}%.".format(100*P_ill_P))

The probability that the patient is ill on condition that they were positively diagnosed is: 8.799%.


# 3.Expected Value Problem

a).Let $g(X) = 1$ for some set $A$ being a subset of sample space $\Omega$. What is the interpretation of $E[g(X)]$ if is discrete with a given PMF or continuous with a given PDF?

In order to solve this problem we have to make an assumption about the values of $g(x)$ for $x\notin A$. Since the author of this question did not specify what happens for x from within $\Omega$ but outside of $A$ region we have taken the liberty to specify function $g(x)$ just a tap more

$$g(x)= 
     \begin{cases}
      &1, x\in A\\
      &0, x\notin A
     \end{cases}.$$
     
For continuous $x \in A$:
$$E\left[g(X)\right]=\int_{\Omega}g(x)\cdot f_{x}(x) dx=\int_{\Omega} f_{x}(x) dx = 1,$$

where in accordance to the lecture $f_{x}(x)$ is PDF function.  

For discrete $k\in A$ and function $g(x)$ defined in the aforementioned way it is only natural to conclude that:

$$E\left[g(X)\right]=\int_{\Omega}g(x)\cdot f_{x}(x) \cdot \delta(x-k) dx=\sum_{k\in A}g(k)\cdot p_{x}(k)=\sum_{k\in A}p_{x}(k)=1,$$

where $p_{x}(k)$ is the PMF function.

The above equation can be trivially generalised to $n$ dimensions: $$x\rightarrow (x_{1},x_{2},.....,x_{n}),$$ $$dx\rightarrow dx_{1}dx_{2}...dx_{n}.$$

b).What is the interpretation of $E\left[g(X)\right]$ for $g(X)=x$?


$$E\left[g(X)\right]=\int_{\Omega}g(x)\cdot f_{x}(x) dx=\int_{\Omega}x\cdot f_{x}(x) dx.$$

